In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("HM2-1").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

23/11/21 10:40:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from pyspark import SparkFiles
from pyspark.sql.types import *
from datetime import datetime
import pandas as pd

custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

news_data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00432/Data/News_Final.csv", parse_dates=['PublishDate'], date_parser=custom_date_parser)
display(news_data)

/tmp/ipykernel_12/3720108920.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  news_data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00432/Data/News_Final.csv", parse_dates=['PublishDate'], date_parser=custom_date_parser)


,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
0,99248.0,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,0.000000,-0.053300,-1,-1,-1
1,10423.0,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,0.208333,-0.156386,-1,-1,-1
2,18828.0,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-0.425210,0.139754,-1,-1,-1
3,27788.0,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,0.000000,0.026064,-1,-1,-1
4,27789.0,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,0.000000,0.141084,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
93234,61851.0,Stocks rise as investors key in on US economy ...,The June employment report is viewed as a cruc...,MarketWatch,economy,2016-07-07 15:31:05,0.104284,0.044943,-1,3,5
93235,61865.0,Russian PM proposes to use conservative and to...,"In addition, establish stimulating economic po...",TASS,economy,2016-07-07 15:31:10,0.072194,0.000000,-1,0,1
93236,104793.0,Palestinian Government Uses Foreign Aid To Pay...,The Palestinian government spends nearly $140 ...,Daily Caller,palestine,2016-07-07 15:38:26,0.291667,-0.139754,5,1,0
93237,104794.0,Palestine Youth Orchestra prepares for first U...,Palestine Youth Orchestra prepares for first U...,Ahram Online,palestine,2016-07-07 15:59:22,0.121534,0.092313,0,0,0


In [3]:
news_data_schema = StructType([StructField("IDLink", FloatType(), True)\
                               ,StructField("Title", StringType(), True)\
                               ,StructField("Headline", StringType(), True)\
                               ,StructField("Source", StringType(), True)\
                               ,StructField("Topic", StringType(), True)\
                               ,StructField("PublishDate", TimestampType(), True)\
                               ,StructField("SentimentTitle", FloatType(), True)\
                               ,StructField("SentimentHeadline", FloatType(), True)\
                               ,StructField("Facebook", IntegerType(), True)\
                               ,StructField("GooglePlus", IntegerType(), True)\
                               ,StructField("LinkedIn", IntegerType(), True)])

news_data = spark.createDataFrame(news_data, schema = news_data_schema)
news_data.show(n=10)

/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/types.py:238: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):
23/11/21 10:40:38 WARN TaskSetManager: Stage 0 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+-------+--------------------+--------------------+--------------------+---------+-------------------+--------------+-----------------+--------+----------+--------+
| IDLink|               Title|            Headline|              Source|    Topic|        PublishDate|SentimentTitle|SentimentHeadline|Facebook|GooglePlus|LinkedIn|
+-------+--------------------+--------------------+--------------------+---------+-------------------+--------------+-----------------+--------+----------+--------+
|99248.0|Obama Lays Wreath...|Obama Lays Wreath...|           USA TODAY|    obama|2002-04-02 00:00:00|           0.0|      -0.05330018|      -1|        -1|      -1|
|10423.0|A Look at the Hea...|Tim Haywood, inve...|           Bloomberg|  economy|2008-09-20 00:00:00|    0.20833333|      -0.15638581|      -1|        -1|      -1|
|18828.0|Nouriel Roubini: ...|Nouriel Roubini, ...|           Bloomberg|  economy|2012-01-28 00:00:00|   -0.42521003|       0.13975425|      -1|        -1|      -1|
|27788.0|F

In [4]:
import string
import re
import pyspark.sql.functions as func

#Remove punctuations and extra white space
def remove_punctuations(text):
    text = text.lower()
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    text = re.sub("\s+", " ", text)
    text = text.strip()
    return text

udf_remove_punctuations = func.udf(lambda x:remove_punctuations(x),StringType())
news_data = news_data.withColumn('Headline', udf_remove_punctuations(func.col('Headline')))
news_data = news_data.withColumn('Title', udf_remove_punctuations(func.col('Title')))
news_data.select('Title').show(5, False)

23/11/21 10:40:42 WARN TaskSetManager: Stage 1 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------------------------------------------------+
|Title                                              |
+---------------------------------------------------+
|obama lays wreath at arlington national cemetery   |
|a look at the health of the chinese economy        |
|nouriel roubini global economy not back to 2008    |
|finland gdp expands in q4                          |
|tourism govt spending buoys thai economy in january|
+---------------------------------------------------+
only showing top 5 rows



#### Words count in two fields: ‘Title’ and ‘Headline’ respectively and list of the most frequent words according to the term frequency in descending order, in total, per day, and per topic, respectively

In [5]:
#Most frequent words in total for the column Title

news_data.withColumn('word', func.explode(func.split(func.col('Title'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show(20, False)

23/11/21 10:40:43 WARN TaskSetManager: Stage 2 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------+-----+
|word     |count|
+---------+-----+
|obama    |27990|
|to       |27545|
|economy  |26710|
|s        |23111|
|microsoft|20690|
|the      |17785|
|in       |16007|
|for      |12514|
|of       |11463|
|on       |10791|
|and      |7833 |
|a        |7721 |
|is       |6669 |
|with     |5425 |
|palestine|4525 |
|as       |4231 |
|us       |3873 |
|new      |3778 |
|says     |3522 |
|at       |3521 |
+---------+-----+
only showing top 20 rows



In [6]:
#Most frequent words per day in the column Title

news_data.withColumn('word', func.explode(func.split(func.col('Title'), ' ')))\
    .groupBy(func.date_format('PublishDate','yyyy-MM-dd').alias('day'), 'word')\
    .count()\
    .sort('count', ascending=False)\
    .show(20, False)

23/11/21 10:40:54 WARN TaskSetManager: Stage 4 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+----------+---------+-----+
|day       |word     |count|
+----------+---------+-----+
|2016-06-13|microsoft|314  |
|2016-01-12|obama    |287  |
|2016-03-10|obama    |266  |
|2016-03-16|obama    |255  |
|2016-01-13|obama    |255  |
|2015-12-18|obama    |252  |
|2016-06-13|to       |239  |
|2016-02-09|obama    |233  |
|2016-03-16|to       |229  |
|2016-01-12|to       |211  |
|2015-11-24|obama    |211  |
|2016-01-05|obama    |208  |
|2016-01-12|the      |207  |
|2016-01-13|s        |203  |
|2015-12-01|obama    |202  |
|2016-03-15|obama    |201  |
|2016-01-28|microsoft|200  |
|2016-03-30|microsoft|200  |
|2016-03-10|to       |200  |
|2016-03-30|to       |198  |
+----------+---------+-----+
only showing top 20 rows



In [7]:
#Most frequent words per Topic in the column Title

news_data.withColumn('word', func.explode(func.split(func.col('Title'), ' ')))\
    .groupBy('Topic', 'word')\
    .count()\
    .sort('count', ascending=False)\
    .show(20, False)

23/11/21 10:40:59 WARN TaskSetManager: Stage 6 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------+---------+-----+
|Topic    |word     |count|
+---------+---------+-----+
|obama    |obama    |27547|
|economy  |economy  |26586|
|microsoft|microsoft|20677|
|obama    |to       |10275|
|economy  |s        |9112 |
|economy  |the      |8983 |
|economy  |to       |8850 |
|obama    |s        |8139 |
|economy  |in       |6881 |
|microsoft|to       |6319 |
|microsoft|s        |4815 |
|palestine|palestine|4523 |
|obama    |the      |4141 |
|obama    |in       |4096 |
|economy  |of       |4065 |
|obama    |on       |4045 |
|economy  |for      |3986 |
|obama    |for      |3727 |
|economy  |on       |3717 |
|microsoft|for      |3671 |
+---------+---------+-----+
only showing top 20 rows



In [8]:
#Most frequent words in total for the column Headline

news_data.withColumn('word', func.explode(func.split(func.col('Headline'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show(20, False)

23/11/21 10:41:02 WARN TaskSetManager: Stage 8 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------+------+
|word     |count |
+---------+------+
|the      |155341|
|to       |69878 |
|of       |61664 |
|a        |60868 |
|in       |54624 |
|and      |48534 |
|s        |46430 |
|obama    |30900 |
|on       |30160 |
|is       |26907 |
|economy  |26342 |
|for      |26016 |
|that     |24912 |
|microsoft|24852 |
|president|23364 |
|has      |17011 |
|with     |16510 |
|quot     |16024 |
|as       |15614 |
|it       |14513 |
+---------+------+
only showing top 20 rows



In [9]:
#Most frequent words per day in the column Headline

news_data.withColumn('word', func.explode(func.split(func.col('Headline'), ' ')))\
    .groupBy(func.date_format('PublishDate','yyyy-MM-dd').alias('day'), 'word')\
    .count()\
    .sort('count', ascending=False)\
    .show(20, False)

23/11/21 10:41:06 WARN TaskSetManager: Stage 10 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+----------+----+-----+
|day       |word|count|
+----------+----+-----+
|2016-04-14|the |1199 |
|2016-01-12|the |1177 |
|2015-12-01|the |1127 |
|2016-06-13|the |1105 |
|2015-11-24|the |1100 |
|2016-03-10|the |1088 |
|2016-01-13|the |1085 |
|2016-03-16|the |1076 |
|2016-06-24|the |1056 |
|2016-01-28|the |1013 |
|2016-03-15|the |990  |
|2015-11-30|the |973  |
|2016-03-01|the |958  |
|2016-01-19|the |942  |
|2015-12-22|the |935  |
|2016-03-31|the |932  |
|2016-01-05|the |929  |
|2016-04-28|the |921  |
|2016-05-31|the |914  |
|2016-03-30|the |910  |
+----------+----+-----+
only showing top 20 rows



In [10]:
#Most frequent words per Topic in the column Headline

news_data.withColumn('word', func.explode(func.split(func.col('Headline'), ' ')))\
    .groupBy('Topic', 'word')\
    .count()\
    .sort('count', ascending=False)\
    .show(20, False)

23/11/21 10:41:11 WARN TaskSetManager: Stage 12 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------+---------+-----+
|Topic    |word     |count|
+---------+---------+-----+
|economy  |the      |67182|
|obama    |the      |43426|
|obama    |obama    |30422|
|microsoft|the      |30135|
|economy  |economy  |26041|
|microsoft|microsoft|24836|
|obama    |to       |24426|
|economy  |of       |23387|
|economy  |in       |22674|
|economy  |to       |22444|
|obama    |president|21149|
|economy  |a        |20981|
|economy  |s        |19165|
|obama    |a        |18466|
|economy  |and      |18046|
|microsoft|to       |17561|
|obama    |of       |17405|
|obama    |in       |16273|
|microsoft|a        |15202|
|obama    |s        |14994|
+---------+---------+-----+
only showing top 20 rows



#### Sum and average sentiment score of each topic, respectively

In [11]:
#Sum and average sentiment of each topic
news_data.groupBy('Topic').agg(func.sum(news_data.SentimentTitle ),func.avg(news_data.SentimentTitle)).show()

23/11/21 10:41:14 WARN TaskSetManager: Stage 14 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------+-------------------+--------------------+
|    Topic|sum(SentimentTitle)| avg(SentimentTitle)|
+---------+-------------------+--------------------+
|microsoft|  51.56650673694094|0.002359159426157...|
|  economy|-350.89723831791343|-0.01034240858046...|
|    obama|-27.604435646469938|-9.64852696486191...|
|palestine|-177.61514392797835|-0.02008539454121...|
+---------+-------------------+--------------------+



#### Top-100 frequent words and their co-occurrence matrices (100x100), per topic in titles and headlines respectively. Each entry in the matrix contain the co-occurrence frequency in all news titles and headlines, respectively.

In [12]:
#Most frequent words in total for the column Title
f = news_data.withColumn('word', func.explode(func.split(func.col('Title'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)
f.show()

f.select('word').head(100)

23/11/21 10:41:16 WARN TaskSetManager: Stage 24 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:41:18 WARN TaskSetManager: Stage 26 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------+-----+
|     word|count|
+---------+-----+
|    obama|27990|
|       to|27545|
|  economy|26710|
|        s|23111|
|microsoft|20690|
|      the|17785|
|       in|16007|
|      for|12514|
|       of|11463|
|       on|10791|
|      and| 7833|
|        a| 7721|
|       is| 6669|
|     with| 5425|
|palestine| 4525|
|       as| 4231|
|       us| 3873|
|      new| 3778|
|     says| 3522|
|       at| 3521|
+---------+-----+
only showing top 20 rows



[Row(word='obama'),
 Row(word='to'),
 Row(word='economy'),
 Row(word='s'),
 Row(word='microsoft'),
 Row(word='the'),
 Row(word='in'),
 Row(word='for'),
 Row(word='of'),
 Row(word='on'),
 Row(word='and'),
 Row(word='a'),
 Row(word='is'),
 Row(word='with'),
 Row(word='palestine'),
 Row(word='as'),
 Row(word='us'),
 Row(word='new'),
 Row(word='says'),
 Row(word='at'),
 Row(word='president'),
 Row(word='will'),
 Row(word='economic'),
 Row(word='china'),
 Row(word='windows'),
 Row(word='2016'),
 Row(word='10'),
 Row(word='from'),
 Row(word='global'),
 Row(word='by'),
 Row(word='t'),
 Row(word='trump'),
 Row(word='be'),
 Row(word='about'),
 Row(word='up'),
 Row(word='growth'),
 Row(word='not'),
 Row(word='how'),
 Row(word='it'),
 Row(word='over'),
 Row(word='more'),
 Row(word='palestinian'),
 Row(word='israel'),
 Row(word='after'),
 Row(word='its'),
 Row(word='are'),
 Row(word='state'),
 Row(word='world'),
 Row(word='what'),
 Row(word='why'),
 Row(word='has'),
 Row(word='but'),
 Row(word='ca

In [15]:
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window

def cooccurrence(topic, column, n):
    top_n = news_data[news_data["Topic"] == topic].withColumn('word', func.explode(func.split(func.col(column), ' ')))\
        .groupBy('word')\
        .count()\
        .sort('count', ascending=False)\
        .limit(n)
    top_n = list(top_n.select(top_n.word).toPandas()['word'])
    
    long = news_data[news_data["Topic"] == topic].withColumn("id", func.monotonically_increasing_id())\
    .select("id", func.explode(func.split(func.col(column), ' ')))
    #long.show()
    
    long1 = long.filter(long.col.isin(top_n))
    #long1.show()
    #print(long1.count())
    
    long1.filter(long1.col.isin(top_n)).withColumnRenamed("col", "col_")\
        .join(long1, ["id"])\
        .stat.crosstab("col_", "col")\
        .show()
    

In [16]:
cooccurrence("microsoft","Title", 5)

23/11/21 10:42:23 WARN TaskSetManager: Stage 30 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:42:26 WARN TaskSetManager: Stage 32 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:42:27 WARN TaskSetManager: Stage 33 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------+----+---------+----+----+----+
| col__col| for|microsoft|   s| the|  to|
+---------+----+---------+----+----+----+
|      for|3831|     3499| 651| 609| 638|
|        s| 651|     4621|5473| 796|1121|
|       to| 638|     6032|1121| 835|7463|
|microsoft|3499|    21011|4621|3092|6032|
|      the| 609|     3092| 796|3876| 835|
+---------+----+---------+----+----+----+



In [17]:
cooccurrence("microsoft","Title", 100)

23/11/21 10:42:37 WARN TaskSetManager: Stage 52 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:42:41 WARN TaskSetManager: Stage 54 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:42:42 WARN TaskSetManager: Stage 55 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:42:52 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+---+---+---+----+---+---+---+---+---+-----+-----+------+---+---+-------+---------+---+-----+----+---+---+---+-----+----+---+---+-----+--------+---+---+---+---+-----+-----------+-------+----+----+--------+-----+----+----+----+------+---+----+--------+---+----+----+---+----+---+---+--------+--------+-----+---------+------+----+----+-------+---+---+---+---+------+---+---+----+---+----+---+-----+---+----+----+----+--------+-----+-----+-------+-------+---+----+----+---+----+----+---+------+-------+-----+----+---+----+-------+----+----+---+----+
|col__col|  1| 10|  2|2016|  3|365|  4|  5|  a|about|after|amazon| an|and|android|announces|app|apple|apps|are| as| at|azure|band| be|big|build|business|but| by|can|ceo|cloud|corporation|cortana|data|edge|facebook|first| for|free|from|google|has|here|hololens|how|  in|into|ios|  is| it|its|launches|linkedin|lumia|microsoft|mobile|more|msft|nadella|new|now| of|off|office| on|one|open|out|over| pc|phone|pro|quot|   s|says|security|skype|store

In [18]:
cooccurrence("economy","Title", 100)

23/11/21 10:42:52 WARN TaskSetManager: Stage 74 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:42:55 WARN TaskSetManager: Stage 76 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:42:55 WARN TaskSetManager: Stage 77 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+--------+---+---+---+----+----+---+---+---+---+---+----+-----+-----+----+---+----+---+----+---+-----+------+--------+---+---+---+------+-----+-------+-----+----+-------+--------+-------+---+---+-----+----+----+---+------+----+----+-------+-----+------+---+----+---+---+----+-----+----+---+---+-----+----+---+------+---+--------+----+---+---+---+-----+----+---+----+----+-------+-------+----+----+---------+------+------+-------+-----+----+-------+-----+-----+---+----+----+----+----+----+-----+---+---+---+----+----+---+----+----+-----+-----+----+
|col__col|  0|  1|  2|2015|2016|  3|  4|  5|  6|  7|   a|about|after| and|are|  as| at|bank| be|boost|brexit|business|but| by|can|canada|china|chinese|could|data|despite|economic|economy| eu|fed|first| for|from|gdp|global|good|grow|growing|grows|growth|has|help|how|imf|  in|india|  is| it|its|japan|jobs|low|market|may|minister|more|new| no|not|obama|  of|oil|  on|over|percent|quarter|quot|rate|recession|report|russia|russian|    s|says|sharing|sta

In [19]:
cooccurrence("obama","Title", 100)

23/11/21 10:43:03 WARN TaskSetManager: Stage 96 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:06 WARN TaskSetManager: Stage 98 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:06 WARN TaskSetManager: Stage 99 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+-----------+----+---+-----+-------+--------------+-----+-------+-------+---+---+---+---+---+------+---+----+---+---+-----+--------+---+------+-------+-------+--------+-------+-----+----+----+------+-----+-----+----+----+---+---+---+----+---+-------+---------+---+-----+---+---+-----------+----+----+----+----+-------+---+----+-------+------+----+----+--------+----+---+---+---+-----+----+----+---+----+----+---------+----+-----------+----+-------+----+------+-----+-------+-----+---+------+----+----+----+----+----+-----+---+---+---+-----+---+---+----+-----+---+---+----+----+-----+----+
|   col__col|2016|  a|about|address|administration|after|against|america| an|and|are| as| at|barack| be|bill|but| by|calls|campaign|can|change|climate|clinton|congress|control|court|cuba|deal|donald|fight|first| for|from|gop|gun|has|have| he|hillary|hiroshima|his|house|how|  i|immigration|  in|iran|  is|isis|islamic| it|just|leaders|legacy|make|meet|michelle|more|new| no|not|obama|  of|  on|out|over|plan|pre

In [20]:
cooccurrence("palestine","Title", 100)

23/11/21 10:43:13 WARN TaskSetManager: Stage 118 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:15 WARN TaskSetManager: Stage 120 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:16 WARN TaskSetManager: Stage 121 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+-----------+---+----+----+---+-----+-----+-----+-------+---+---+---+----+----+---+---+----+---+---+-----+--------+--------+---+---+----+----+---+-----+-----+--------+----+------+------+----+----+-----+---+----+-------------+----+---+------+-------+---+---------+------+-------+------+---+---+----+------+---+---+---+---+----------+---+---+---+----+---------+-----------+------------+-----+------+---------+---+----+------+------+----+----+------+----------+--------+-----+--------+-------+------+-----+---+-----+-----+---+----+---+-----+-----+---+---+---+---+-----+----+----+---+----+----+-----+-----+
|   col__col|  1|2015|2016|  a|abbas|about|after|against| al| an|and|anti|arab| as| at|bank| be| by|calls|children|conflict|day| de|down|east| eu|first|flood|flooding| for|forces|france|from|gaza|hamas|how|  in|international|into| is|israel|israeli| it|jerusalem|jewish|justice|killed|  l| la|land|leader|man|new| no|not|occupation| of| on|one|over|palestine|palestinian|palestinians|peace|police|

In [21]:
cooccurrence("microsoft","Headline", 5)

23/11/21 10:43:22 WARN TaskSetManager: Stage 140 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:24 WARN TaskSetManager: Stage 142 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:25 WARN TaskSetManager: Stage 143 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------+-----+---------+-----+-----+-----+
| col__col|    a|microsoft|   of|  the|   to|
+---------+-----+---------+-----+-----+-----+
|        a|26888|    18830|12573|24274|15144|
|       to|15144|    21940|12669|30401|33967|
|microsoft|18830|    35156|16732|37207|21940|
|       of|12573|    16732|23089|27267|12669|
|      the|24274|    37207|27267|80245|30401|
+---------+-----+---------+-----+-----+-----+



In [22]:
cooccurrence("microsoft","Headline", 100)

23/11/21 10:43:28 WARN TaskSetManager: Stage 162 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:30 WARN TaskSetManager: Stage 164 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:31 WARN TaskSetManager: Stage 165 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+--------+---+----+---+----+----+-----+-----+---+----+----+---------+---+-----+----+----+----+-----+----+----+-----+--------+----+----+---+-----+-------+----+----+-------+-----+-----+----+------+----+----+---+-----+----+-----+----+----+----+----+------+----+-----+---------+------+----+----+----+---+---+----+------+----+----+---+---+----+--------+----+----+--------+-----+----+--------+--------+----+-------+----+----+----------+----+----+-----+-----+----+----+----+----+-----+---+----+------+-----+-------+----+---+----+----+----+-----+----+-------+----+----+----+----+----+
|col__col|  1|  10|  2|2016|   a|about|after|all|  an| and|announced|app|apple| are|  as|  at|azure|  be|been|build|business| but|  by|can|cloud|company|corp|data|devices|first|  for|from|google| has|have| if|   in|into|   is|  it| its|just|last|latest|like|lumia|microsoft|mobile|more|msft| new|not|now|  of|office|  on| one| or|out|over|platform| pro|quot|released|    s|said|services|software|some|surface|   t|tech|tech

In [23]:
cooccurrence("economy","Headline", 100)

23/11/21 10:43:39 WARN TaskSetManager: Stage 184 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:41 WARN TaskSetManager: Stage 186 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:42 WARN TaskSetManager: Stage 187 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------+----+----+----+----+----+----+----+----+-----+-----+---------+-----+----+-----+----+----+----+----+----+----+--------+----+----+-----+-----+-------+----+--------+-------+--------+--------+-------+---------+-----+----+----+------+----------+----+------+----+----+----+----+----+-----+----+-----+----+----+----+----+------+----+--------+----+----+----+----+-----+----+----+----+----+----+-------+---------+------+-------+----+----+------+-----+----+----+-----+---+----+----+-----+-----+-----+----+----+-----+-----+----+----+----+----+----+----+-----+-----+----+----+-----+-----+----+-----+
| col__col|   0|   1|   2|2015|2016|   3|   5|   7|    a|about|according|after|  an|  and| are|  as|  at|bank|  be|been|business| but|  by|china|could|country|data|economic|economy|european|expected|federal|financial|first| for|from|global|government|grew|growth| has|have|  he| his|  if|   in|into|   is|  it| its|jobs|last|market| may|minister|more|most| new| not|   of| oil|  on| one|over| per|perc

In [24]:
cooccurrence("obama","Headline", 100)

23/11/21 10:43:49 WARN TaskSetManager: Stage 206 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:51 WARN TaskSetManager: Stage 208 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:43:52 WARN TaskSetManager: Stage 209 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+---------+-----+-----+-------+--------------+-----+-------+--------+----+-----+----+----+----+----+------+----+----+----+----+-------+--------+-----+------+------+-----+-----+----+------+----+---+---+----+----+----+-------+-----+-----+---+-----+----+----+----+---+----+------+----+--------+----+----+-----+-----+------+----+----+---+----+------+---------+------------+----+----------+-----+----+----+-----+------+-------+---+----+----+-----+-----+----+----+--------+----+-----+-----+-------+---+----+------+---+----+-----+----+----------+---+---------+----+----+----+-----+-----+----+----+----+-----+-----+----+-----+
| col__col|    a|about|address|administration|after|against|american|  an|  and|  ap| are|  as|  at|barack|  be|been| but|  by|clinton|congress|court|donald|during|final|first| for|friday|from|gun|had| has|have|  he|hillary|  his|house|  i|   in|into|  is|  it|its|last|monday|more|national| new| not|obama|   of|office|  on| one|out|over|people|president|presidential|quot|republi

In [25]:
cooccurrence("palestine","Headline", 100)

23/11/21 10:44:00 WARN TaskSetManager: Stage 228 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:44:02 WARN TaskSetManager: Stage 230 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.
23/11/21 10:44:02 WARN TaskSetManager: Stage 231 contains a task of very large size (6245 KiB). The maximum recommended task size is 1000 KiB.


+-----------+----+----+----+-----+-----+---------+-----+-------+---+---+---+----+---+---+---+---------+----+---+----+-------+---+---+----+--------+---+------+----+-----+----+------+----+----+---+---+----+---+---+---+---+----+-------------+----+------+-------+---+---+---------+----+-------+-----+------+--------+------+----+-------+---+---+--------+----+---+----+---+----+---------+-----------+------------+-----+------+------+---------+----+--------+----+----+-----+-----+----+-----+-----+----+----+--------+----+-------+---+------+---+----+---+---------+----+----+----+-----+---+----+----+-----+----+-----+
|   col__col|2015|2016|   a|abbas|about|according|after|against| al|all| an| and|are| as| at|authority|bank| be|been|between|but| by|city|conflict| de|during|east|first| for|friday|from|gaza|had|has|have| he|her|his|  i|  in|international|  is|israel|israeli| it|its|jerusalem|last|mahmoud|media|middle|minister|monday|more|nations|new|not|occupied|  of|old|  on|one|over|palestine|palestini